# HW7: Beam Search Decoding - News Headline Generation

In this exercise, you are going to learn and implement decoding techniques for sequence generation. Usually, the sequence is generated word-by-word from a model. In each step, the model predicted the most likely word based on the predicted words in previous steps (this is called auto-regressive decoding).

As such, it is very important how you decide on what to predicted at each step, as it will be conditioned on to predicted all of the following steps. We will implement two of main decoding techniques introduced in the lecture: **Greedy Decoding** and **Beam Search Decoding**. Greedy Decoding immediately chooses the word with best score at each step, while Beam Search Decoding focuses on the sequence that give the best score overall.

To complete this exercise, you will need to complete the methods for decoding for a text generation model trained on [New York Times Comments and Headlines dataset](https://www.kaggle.com/aashita/nyt-comments). The model is trained to predict a headline for the news given seed text. You do not need to train any model model in this exercise as we provide both the pretrained model and dictionary.


## Download model and vocab and setup

In [69]:
# !wget -O vocab.txt https://www.dropbox.com/s/ht12ua9vpkep6l8/hw9_vocab.txt?dl=0
# !wget -O model.bin https://www.dropbox.com/s/okmri7cnd729rr5/hw9_model.bin?dl=0

In [70]:
# import torch
# import torch.nn as nn
# from tokenizers import Tokenizer
# from tokenizers.models import WordLevel
# from tokenizers.pre_tokenizers import Whitespace
# import numpy as np

In [71]:
class RNNmodel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dropout_rate):

        super().__init__()
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, 128, num_layers=2,
                     batch_first=True)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, vocab_size)

    def forward(self, src):
        embedding = self.embedding(src)
        output,_ = self.rnn(embedding)
        output = self.dropout(output)
        prediction = self.fc2(output)
        return prediction

In [72]:
with open("vocab.txt") as f:
  vocab_file = f.readlines()
embedding_dim = 64
dropout_rate = 0.2

model = RNNmodel(len(vocab_file), embedding_dim, dropout_rate)
model.load_state_dict(torch.load("model.bin",map_location='cpu'))
model.eval()

<ipython-input-72-e56f35018f72>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model.bin",map_location='cpu'))


RNNmodel(
  (embedding): Embedding(10054, 64)
  (rnn): LSTM(64, 128, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=128, out_features=10054, bias=True)
)

In [73]:
vocab = [v.strip() for v in vocab_file]
vocab_size = len(vocab)
print(f"Vocab Size: {vocab_size}")
vocab[:10]

Vocab Size: 10054


['<unk>', '<pad>', '<eos>', 'the', 'a', 'to', 'of', 's', 'in', 'for']

In [74]:
stoi = { ch:i for i,ch in enumerate(vocab) }
tokenizer = Tokenizer(WordLevel(stoi, unk_token="<unk>"))
tokenizer.pre_tokenizer = Whitespace()
tokenized_text = tokenizer.encode("the a of to unknowns")
print(tokenized_text)
print(tokenized_text.ids)
print(tokenized_text.tokens)
print(tokenizer.decode(tokenized_text.ids))

Encoding(num_tokens=5, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
[3, 4, 6, 5, 0]
['the', 'a', 'of', 'to', '<unk>']
the a of to <unk>


## 1. TODO: Greedy decode
Normally, in sequence generation task, the model will continue generating tokens until an end-of-sequence symbol appear or the maximum length is reached. For this task:
- The end-of-sequence symbol is "< eos >" and its index is 2
- Use the maximum generation length of 15

In [75]:
eos_token = '<eos>'
max_gen_length = 15

In [76]:
def greedy_decode(seed_text, tokenizer):
    """Greedy decodes with seed text.

        Args:
        seed_text: The seed string to be used as initial input to the model.
        tokenizer: The tokenizer for converting word to index and back.

        Your code should do the followings:
          1. Convert current_text to sequences of indices
          2. Predict the next token using the model and choose the token with the highest score as output
          3. Append the predicted index to current_text
          4. Loop until completion
          5. Return text prediction and a list of probabilities of each step

        You do not need to stop early when end-of-sequence token is generated and can continue decoding
        until max_gen_length is reached. We can filter the eos token out later.
    """
    probs = []
    is_eos = False
    output = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    while len(output[0]) < max_gen_length:
        prediction = model(output)
        pred_probs = nn.functional.softmax(prediction[0][-1],dim=0)
        max_word_p_idx = torch.argmax(pred_probs)
        max_word_p_idx = torch.tensor([max_word_p_idx])
        if(int(max_word_p_idx[0]) == vocab.index("<eos>")):
            output = torch.cat((output, torch.tensor([[max_word_p_idx]])), dim=1)
            is_eos = True
        if not is_eos:
            probs.append(float(pred_probs[max_word_p_idx[0]]))
        output = torch.cat((output, torch.tensor([[max_word_p_idx]])), dim=1)
    output=output[0].tolist()
    return output,probs

In [77]:
def clean_output(text_ids, eos_token_id):
    """Drop eos_token and every words that follow"""
    if eos_token_id in text_ids:
        index = text_ids.index(eos_token_id)
        text_ids = text_ids[:index]
    return tokenizer.decode(text_ids)

In [78]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
for seed in sample_seeds:
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi["<eos>"])
    print("Seed:", seed)
    print("Output:",cleaned_text)
    print("probs:",probs)
    print("\n")

Seed: to
Output: to encourage creativity in the new york bill
probs: [0.04071051999926567, 0.2661307156085968, 0.25815948843955994, 0.2311648279428482, 0.07794329524040222, 0.06978018581867218, 0.2244524359703064]


Seed: america
Output: america s lethal export
probs: [0.44965454936027527, 0.04639991372823715, 0.7206957340240479]


Seed: people
Output: people to balloon to make a criminal with a dog with a callous rival
probs: [0.35848477482795715, 0.03946574404835701, 0.26953017711639404, 0.033772923052310944, 0.28156694769859314, 0.030896717682480812, 0.3262561559677124, 0.32244035601615906, 0.03489115461707115, 0.2229832410812378, 0.580828845500946, 0.04674474895000458, 0.26616525650024414]


Seed: next
Output: next phenom english clubs 2 call another deal in the same arrivals
probs: [0.1724235713481903, 0.23235632479190826, 0.7396268248558044, 0.0788838192820549, 0.044043004512786865, 0.057693466544151306, 0.032969508320093155, 0.3304014205932617, 0.6040374040603638, 0.031745906919

Your output should be:

*   to encourage creativity in the new york bill
*   america s lethal export
*   people to balloon to make a criminal with a dog with a callous rival
*   next phenom english clubs 2 call another deal in the same arrivals
*   picture perfect chapter a spot of view of banning care  
*   on the catwalk in saudi arabia







## 2. TODO: Beam search decode

Another well-known decoding method is beam search decoding that focuses more on the overall sequence score.

Instead of greedily choosing the token with the highest score for each step, beam search decoding expands all possible next tokens and keeps the __k__ most likely sequence at each step, where __k__ is a user-specified beam size. A sequence score is also calculated according user-specified cal_score() function.
The beam with the highest score after the decoding process is done will be the output.

There are a few things that you need to know before implementing a beam search decoder:
- When the eos token is produced, you can stop expanding that beam
- However, the ended beams must be sorted together with active beams
- The decoding ends when every beams are either ended or reached the maximum length, but for this task, you can continue decoding until the max_gen_len is reached
- We usually work with probability in log scale to avoid numerical underflow. You should use np.log(score) before any calculation
- **As probabilities for some classes will be very small, you must add a very small value to the score before taking log e.g np.log(prob + 0.00000001)**

#### Sequence Score
The naive way to calculate the sequence score is to __multiply every token scores__ together. However, doing so will make the decoder prefer shorter sequence as you multiply the sequence score with a value between \[0,1\] for every tokens in the sequence. Thus, we usually normalize the sequence score with its length by calculating its __geometric mean__ instead.

**You should do this in log scale**

In [79]:
def cal_score(score_list, length, normalized=False): #cal score for each beam from a list of probs
    log_scores = [np.log(score+0.00000001) for score in score_list]
    if normalized:
         score = sum(log_scores) / length
    else:
         score = sum(log_scores) 
    return score

In [80]:
# sample_seeds = ["to", "america", "people", "next", "picture", "on"]
# for seed in sample_seeds:
#     output_text, probs = greedy_decode(seed, tokenizer)
#     cleaned_text = clean_output(output_text, stoi["<eos>"])
#     print("Seed:", seed)
#     print("Output:",cleaned_text)
#     print("probs:",probs)
#     print("\n")

In [81]:
def sort_by_norm(x):
    return cal_score(x[1], x[2], True)

def sort_by_unnorm(x):
    return cal_score(x[1], x[2], False)

def beam_search_decode(seed_text, max_gen_len, tokenizer, beam_size=5, normalized=False):
    """We will do beam search decoing using seed text in this function.

    Output:
    beams: A list of top k beams after the decoding ended, each beam is a list of
      [seed_text, list of scores, length]

    Your code should do the followings:
    1.Loop until max_gen_len is reached.
    2.During each step, loop thorugh each beam and use it to predict the next word.
      If a beam is already ended, continues without expanding.
    3.Sort all hypotheses according to cal_score().
    4.Keep top k hypotheses to be used at the next step.
    """
    # For each beam we will store (generated text, list of scores, and current length, is_finished)
    # Add initial beam
    seed_ids = torch.Tensor(tokenizer.encode(seed_text).ids).unsqueeze(0).long()
    beams = [[seed_ids, [], 0, False]]
    for _ in range(max_gen_len):
        new_beams = []
        for beam in beams:
            if beam[3]:
                new_beams.append(beam)
                continue
            else:
                prediction = model(beam[0])
                pred_prob = nn.functional.softmax(prediction[0], dim=-1)[-1]
                for token_id, token_prob in enumerate(pred_prob):
                    new_text = torch.cat((beam[0], torch.tensor([[token_id]])), dim=1)
                    new_finished = (token_id == stoi[eos_token])
                    new_score = beam[1] + [token_prob.item()]
                    new_beams.append([new_text,new_score,beam[2]+1,new_finished])
        if(normalized):
            new_beams.sort(key=sort_by_norm ,reverse=True)  
        else:
            new_beams.sort(key=sort_by_unnorm ,reverse=True)
        
        beams = new_beams[:beam_size]
    return beams

In [82]:
# sample_seeds = ["to", "america", "people", "next", "picture", "on"]
# for seed in sample_seeds:
#     output_text, probs = greedy_decode(seed, tokenizer)
#     cleaned_text = clean_output(output_text, stoi["<eos>"])
#     print("Seed:", seed)
#     print("Output:",cleaned_text)
#     print("probs:",probs)
#     print("\n")

## 3. Generate!
Generate 6 sentences based on the given seed texts.

Decode with the provided seed texts with beam_size 5, max_gen_len 10. Compare the results between greedy, normalized, and unnormalized decoding.

Print the result using greedy decoding and top 2 results each using unnormalized and normalized decoing for each seed text.

Also, print scores of each candidate according to cal_score(). Use normalization for greedy decoding.

In [83]:
sample_seeds = ["to", "america", "people", "next", "picture", "on"]
# max_gen_length = 15
# max_gen_length = 10
# sample_seeds = ["on"]
for seed in sample_seeds:
    max_gen_length = 15
    print("-Greedy-")
    output_text, probs = greedy_decode(seed, tokenizer)
    cleaned_text = clean_output(output_text, stoi["<eos>"])
    score = np.exp(cal_score(probs, len(probs), normalized=True))
    print(cleaned_text ,"%.2f"%score)

    max_gen_length = 10
    print("-Unnormalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        if beam[3]:
            score =  np.exp(cal_score(beam[1][:-1], beam[2]-1, normalized=False))
        else:
            score =  np.exp(cal_score(beam[1], beam[2], normalized=False))            
        print(output_text.title(),"%.2f"%score)

    print("-Normalized-")
    beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
    for beam in beams[:2]:
        output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
        if beam[3]:
            score = np.exp(cal_score(beam[1][:-1], beam[2]-1, normalized=True))
        else:
            score = np.exp(cal_score(beam[1], beam[2], normalized=True))
        print(output_text.title(),"%.2f"%score)
    print("\n")

-Greedy-
to encourage creativity in the new york bill 0.13
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.17
To Consult Exploring Recipes Up The Least Of The Week 0.16


-Greedy-
america s lethal export 0.25
-Unnormalized-
America S Lethal Export 0.02
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.25
America S Desert Aisles 0.20


-Greedy-
people to balloon to make a criminal with a dog with a callous rival 0.14
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.13
People To Balloon For A Criminal With Trump 0.13


-Greedy-
next phenom english clubs 2 call another deal in the same arrivals 0.12
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S 

Your output should be:


```
-Greedy-
to encourage creativity in the new york bill  0.13
-Unnormalized-
To Consult Exploring Recipes For New Jersey 0.00
To Consult Exploring Recipes Up The Pacific Northwest 0.00
-Normalized-
To Consult Exploring Recipes Up The Pacific Northwest 0.17
To Consult Exploring Recipes Up The Least Of The Week 0.16

-Greedy-
america s lethal export  0.25
-Unnormalized-
America S Lethal Export 0.02
America S Desert Aisles 0.01
-Normalized-
America S Lethal Export 0.25
America S Desert Aisles 0.20

-Greedy-
people to balloon to make a criminal with a dog with a callous rival  0.14
-Unnormalized-
People To Balloon For A Criminal 0.00
People To Balloon For A Criminal With Trump 0.00
-Normalized-
People To Balloon For A Criminal With A Second Fiddle 0.13
People To Balloon For A Criminal With Trump 0.13

-Greedy-
next phenom english clubs 2 call another deal in the same arrivals  0.12
-Unnormalized-
Next S Blist Revue 0.00
Next Phenom English Clubs 1 A Chance To Be Back 0.00
-Normalized-
Next S Blist Revue 0.14
Next Phenom English Clubs 1 A Chance To Be Back 0.14

-Greedy-
picture perfect chapter a spot of view of banning care  0.08
-Unnormalized-
Picture Perfect Use Coffee 0.00
Picture Korean A Bonanza Of Pancakes 0.00
-Normalized-
Picture Korean A Bonanza Of Contemplation Times Of Trump S Son 0.12
Picture Korean A Bonanza Of Pancakes 0.07

-Greedy-
on the catwalk in saudi arabia  0.19
-Unnormalized-
On The Billboard Chart 0.00
On The Catwalk In Saudi Arabia 0.00
-Normalized-
On The Whole30 Diet Vowing To Eat Smarter Carbs To Be 0.27
On The Whole30 Diet Vowing To Eat Smarter Carbs For Because 0.26

```



# Answer Questions in MyCourseVille!

Use the seed word "usa" to answer questions in MCV.

In [84]:
seed="usa"
max_gen_length = 15
print("-Greedy-")
output_text, probs = greedy_decode(seed, tokenizer)
cleaned_text = clean_output(output_text, stoi["<eos>"])
score = np.exp(cal_score(probs, len(probs), normalized=True))
print(cleaned_text ,"%.2f"%score)

max_gen_length = 10
print("-Unnormalized-")
beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=False)
for beam in beams[:2]:
    output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
    if beam[3]:
        score =  np.exp(cal_score(beam[1][:-1], beam[2]-1, normalized=False))
    else:
        score =  np.exp(cal_score(beam[1], beam[2], normalized=False))            
    print(output_text.title(),"%.2f"%score)

print("-Normalized-")
beams = beam_search_decode(seed, max_gen_length, tokenizer, beam_size=5, normalized=True)
for beam in beams[:2]:
    output_text = clean_output(beam[0][0].tolist(), stoi[eos_token])
    if beam[3]:
        score = np.exp(cal_score(beam[1][:-1], beam[2]-1, normalized=True))
    else:
        score = np.exp(cal_score(beam[1], beam[2], normalized=True))
    print(output_text.title(),"%.2f"%score)
print("\n")

-Greedy-
usa s duty to investigate 0.09
-Unnormalized-
Usa S Duty To Investigate 0.00
Usa S Bleak Season 3 Episode 4 Recap Chicken 0.00
-Normalized-
Usa S Bleak Season 1 Episode 2 Darkness Descends 0.19
Usa S Bleak Season 3 Episode 4 Recap A Seduction 0.18


